In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [2]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [3]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [6]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [7]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 448       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/368 - 0s - loss: 0.6299 - accuracy: 0.6277
Loss: 0.6299033475958783, Accuracy: 0.6277173757553101


In [10]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Train on 1102 samples
Epoch 1/100
  32/1102 [..............................] - ETA: 13s - loss: 0.6677 - accuracy: 0.6250
Epoch 00001: saving model to checkpoints/weights.01.hdf5
1102/1102 [==============================] - 0s 441us/sample - loss: 0.5678 - accuracy: 0.7078
Epoch 2/100
  32/1102 [..............................] - ETA: 0s - loss: 0.4236 - accuracy: 0.8125
Epoch 00002: saving model to checkpoints/weights.02.hdf5
1102/1102 [==============================] - 0s 52us/sample - loss: 0.4841 - accuracy: 0.7858
Epoch 3/100
  32/1102 [..............................] - ETA: 0s - loss: 0.4857 - accuracy: 0.8438
Epoch 00003: saving model to checkpoints/weights.03.hdf5
1102/1102 [==============================] - 0s 60us/sample - loss: 0.4429 - accuracy: 0.8103
Epoch 4/100
  32/1102 [..............................] - ETA: 0s - loss: 0.5143 - accuracy: 0.7188
Epoch 00004: saving model to checkpoints/weights.04.hdf5
1102/1102 [==============================] - 0s 54us/sample - loss: 0.

 800/1102 [====================>.........] - ETA: 0s - loss: 0.2656 - accuracy: 0.9025
Epoch 00032: saving model to checkpoints/weights.32.hdf5
1102/1102 [==============================] - 0s 71us/sample - loss: 0.2622 - accuracy: 0.9074
Epoch 33/100
 992/1102 [==========================>...] - ETA: 0s - loss: 0.2518 - accuracy: 0.9062
Epoch 00033: saving model to checkpoints/weights.33.hdf5
1102/1102 [==============================] - 0s 68us/sample - loss: 0.2590 - accuracy: 0.9074
Epoch 34/100
  32/1102 [..............................] - ETA: 0s - loss: 0.3156 - accuracy: 0.9062
Epoch 00034: saving model to checkpoints/weights.34.hdf5
1102/1102 [==============================] - 0s 47us/sample - loss: 0.2572 - accuracy: 0.9038
Epoch 35/100
  32/1102 [..............................] - ETA: 0s - loss: 0.4070 - accuracy: 0.9062
Epoch 00035: saving model to checkpoints/weights.35.hdf5
1102/1102 [==============================] - 0s 46us/sample - loss: 0.2548 - accuracy: 0.9065
Epoch 36/

Epoch 64/100
  32/1102 [..............................] - ETA: 0s - loss: 0.2316 - accuracy: 0.8750
Epoch 00064: saving model to checkpoints/weights.64.hdf5

Epoch 00064: saving model to checkpoints/weights.64.hdf5
1102/1102 [==============================] - 0s 52us/sample - loss: 0.2091 - accuracy: 0.9183
Epoch 65/100
  32/1102 [..............................] - ETA: 0s - loss: 0.3745 - accuracy: 0.8750
Epoch 00065: saving model to checkpoints/weights.65.hdf5
1102/1102 [==============================] - 0s 51us/sample - loss: 0.2087 - accuracy: 0.9174
Epoch 66/100
  32/1102 [..............................] - ETA: 0s - loss: 0.2410 - accuracy: 0.8750
Epoch 00066: saving model to checkpoints/weights.66.hdf5
1102/1102 [==============================] - 0s 56us/sample - loss: 0.2073 - accuracy: 0.9165
Epoch 67/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1811 - accuracy: 0.9062
Epoch 00067: saving model to checkpoints/weights.67.hdf5
1102/1102 [=====================

Epoch 96/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1658 - accuracy: 0.9062
Epoch 00096: saving model to checkpoints/weights.96.hdf5
 992/1102 [==========================>...] - ETA: 0s - loss: 0.1731 - accuracy: 0.9335
Epoch 00096: saving model to checkpoints/weights.96.hdf5
1102/1102 [==============================] - 0s 66us/sample - loss: 0.1776 - accuracy: 0.9292
Epoch 97/100
  32/1102 [..............................] - ETA: 0s - loss: 0.3864 - accuracy: 0.8750
Epoch 00097: saving model to checkpoints/weights.97.hdf5
1102/1102 [==============================] - 0s 58us/sample - loss: 0.1769 - accuracy: 0.9310
Epoch 98/100
  32/1102 [..............................] - ETA: 0s - loss: 0.1133 - accuracy: 1.0000
Epoch 00098: saving model to checkpoints/weights.98.hdf5
1102/1102 [==============================] - 0s 57us/sample - loss: 0.1744 - accuracy: 0.9338
Epoch 99/100
  32/1102 [..............................] - ETA: 0s - loss: 0.0878 - accuracy: 1.0000
Epo

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/368 - 0s - loss: 0.4577 - accuracy: 0.8641
Loss: 0.45774078433928284, Accuracy: 0.864130437374115


In [12]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/368 - 0s - loss: 0.4591 - accuracy: 0.8587
Loss: 0.459058856834536, Accuracy: 0.8586956262588501


In [13]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [14]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

368/368 - 0s - loss: 0.4591 - accuracy: 0.8587
Loss: 0.459058856834536, Accuracy: 0.8586956262588501
